In [1]:
import sys
sys.path.append('..')

In [2]:
from tqdm import tqdm

from lib.data import load_all_invoices, load_extracted_samples
from lib.ocr import OCRParams, ocr_sample
from lib.text import extract_invoice, extract_total

In [3]:
all_invoices = load_all_invoices()
len(all_invoices)

8411

In [4]:
# load samples
samples = [sample for sample in load_extracted_samples()]
len(samples)

8411

In [5]:
invoice_dict = {}
for invoice in all_invoices:
    invoice_dict[invoice] = None
    
params = OCRParams(
    engine="easyocr",
    grayscale=False,
    threshold=None
)

detected_but_not_found = []

for sample in tqdm(samples):
    text = sample["text"]
    if text == "":
        text = ocr_sample(sample, params, only_cache=True)
    
    invoice = extract_invoice(text)
    total = extract_total(text)

    if invoice is not None and total is not None:
        if invoice in all_invoices:
            invoice_dict[invoice] = total
        else:
            # print(f'invoice detected {invoice} but not found in all_invoices')
            detected_but_not_found.append(invoice)

  5%|▌         | 426/8411 [00:02<00:52, 152.31it/s]

{'filename': '2021-08-03_1.pdf', 'size': 1104551, 'pages': 2, 'text': '', 'images': ['/mnt/c/Users/mlomb/Desktop/ParaiSUR/data-extracted/2021-08-03_1.pdf/0.png', '/mnt/c/Users/mlomb/Desktop/ParaiSUR/data-extracted/2021-08-03_1.pdf/1.png']}


  6%|▌         | 522/8411 [00:03<00:51, 153.36it/s]


KeyboardInterrupt: 

In [ ]:
detected_but_not_found[:10]

In [ ]:
all_totals = [total for total in invoice_dict.values() if total is not None]
avg = sum(all_totals) / len(all_totals)
avg = round(avg, 2)
avg = 10341.62
len(all_totals), len(all_totals) / len(samples), avg

In [ ]:
# write csv
with open("submission.csv", 'w') as f:
    f.write('Invoice Number,Total Charged\n')
    for invoice, total in invoice_dict.items():

        if total == None:
            total = avg

        f.write(f'{invoice},{total}\n')